In [1]:
#  https://www.kaggle.com/loveall/clicks-conversion-tracking
from pulp import *
import pandas as pd
import numpy as np


In [2]:
# Read the first few rows dataset in a Pandas DataFrame
# Read only the nutrition info not the bounds/constraints
df = pd.read_csv("C:/Users/depri/Desktop/optimisation4th quarter/Project/sales-conversion-optimization/KAG_conversion_data.csv")


# Creating the data frame consiting of only xyz_campaign_id as 916 and 1178
df.loc[(df.xyz_campaign_id==916) | (df.xyz_campaign_id==1178)]

# Cleaning the data
df=df.replace([np.inf, -np.inf], np.nan)


In [3]:
# Creating the additional KPI's
df["Click_through_rate"]=(df.Clicks/df.Impressions)*100
df["Cost_Per_click"]=(df.Spent/df.Clicks)
df["Return_on_advertising_spend"] = round(df.Total_Conversion / df.Spent)
df["Total_Con"]= round((df.Total_Conversion+df.Approved_Conversion),2)
df["Cost_Per_Conversion"]=round((df.Spent/df["Total_Con"]),2)
df["Cost_Per_Mile"]=round((df.Spent / df.Impressions) * 1000,2)
df.head()
# Cleaning the data
df=df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)


Objective function is to minimize cost subject to contraints like total conversion(number of people enquired about the product), 
conversion costs(costs per conversion) and converted(approved conversions).

In [4]:

# First, we create a LP problem with the method LpProblemin PuLP.
prob=LpProblem("Optimzation Problem", LpMinimize)

# Then, we need to create bunches of Python dictionary objects with the information we have from the table. 
# The code is shown below,

# Create a list of advertisement id
advertisement_id=list(df.ad_id)
#print(advertisement_id)

# Create a dictionary of cost for all advertisement ids
costs=dict(zip(advertisement_id, df.Spent))
#print(costs)

# Create a dictionary of total conversion for all advertisement ids
total_conversions = dict(zip(advertisement_id,df.Total_Con))

# Create a dictionary of cost per conversion for all advertisement ids
conversion_costs = dict(zip(advertisement_id,df.Cost_Per_Conversion))

# Create a dictionary return on adverstising spend for all advertisement ids
Returns = dict(zip(advertisement_id,df.Return_on_advertising_spend))

# Create a dictionary return on adverstising spend for all advertisement ids
Converted = dict(zip(advertisement_id,df.Approved_Conversion))


In [5]:
# we create a dictionary of advertisement id variables with lower bound =0 and category 
# continuous i.e. the optimization solution can take any real-numbered value greater than zero.
advertising = LpVariable.dicts("Advertisement",advertisement_id,lowBound=0,cat='Continuous')
# the main objective function
prob+=lpSum([costs[i]*advertising[i] for i in advertisement_id])  # Total cost in which a_id company should invest more.

In [6]:
# total conversion
prob += lpSum([total_conversions[f] * advertising[f] for f in advertisement_id]) >= 1, "Total_Conversion minimum"
prob += lpSum([total_conversions[f] * advertising[f] for f in advertisement_id]) <= 100, "TotalConversion maximum"
#Conversion costs
prob += lpSum([conversion_costs[f] * advertising[f] for f in advertisement_id]) >=2, "Conversion Costs minimum"
prob += lpSum([conversion_costs[f] * advertising[f] for f in advertisement_id]) <=4, "Conversion Costs maximum"
# Converted
prob+= lpSum([Converted[f] * advertising[f] for f in advertisement_id]) >=1, "Converted minimum"
prob+= lpSum([Converted[f] * advertising[f] for f in advertisement_id]) <=100, "Converted maximum"

In [7]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [8]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)
        


Advertisement_777105 = 1.0
Advertisement_781354 = 3.6037736


In [12]:
print("The total cost per add that xyz company should spend given the contraints to acquire 1 customer: {}".format(round(value(prob.objective),2)))

The total cost per add that xyz company should spend given the contraints to acquire 1 customer: 2.09


In [20]:
df[(df.ad_id==777105)|(df.ad_id==781354)][["ad_id","Total_Con","Cost_Per_Conversion",
"Approved_Conversion"]]

,ad_id,Total_Con,Cost_Per_Conversion,Approved_Conversion
267,777105,2,0.09,1
388,781354,1,0.53,0


In [11]:
df[df.ad_id==781354]

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion,Click_through_rate,Cost_Per_click,Return_on_advertising_spend,Total_Con,Cost_Per_Conversion,Cost_Per_Mile
388,781354,936,116371,35-39,F,10,6424,1,0.53,1,0,0.015567,0.53,2.0,1,0.53,0.08


In [ ]:
# $2 to acquire one customer.

In [19]:
df[df.ad_id==777105]

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion,Click_through_rate,Cost_Per_click,Return_on_advertising_spend,Total_Con,Cost_Per_Conversion,Cost_Per_Mile
267,777105,936,115615,45-49,M,63,4333,1,0.18,1,1,0.023079,0.18,6.0,2,0.09,0.04
